In [152]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from scipy.sparse import rand as sprand

ratings = pd.read_csv('ratings_small.csv', usecols = ['userId','movieId','rating'])
user=ratings.userId
user = user.values

item = ratings.movieId
item = item.values


#ratings = torch.tensor(ratings.values)
#ratings=ratings.toarray()
print(user)
print(item)
n_users = len(user)
n_items= len(item)


ratings = ratings.pivot(index='userId', columns='movieId', values='rating')
mat = ratings.values
mat[np.isnan(mat)] = 0
#mat=mat.astype(int)
print(mat)

[  1   1   1 ... 671 671 671]
[  31 1029 1061 ... 6365 6385 6565]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [5. 0. 0. ... 0. 0. 0.]]


In [153]:
class MatrixFactorization(torch.nn.Module):
    
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, 
                                               n_factors,
                                               sparse=True)
        print(self.user_factors)
        self.item_factors = torch.nn.Embedding(n_items, 
                                               n_factors,
                                               sparse=True)
        
    def forward(self, user, item):
        return (self.user_factors(user) * self.item_factors(item)).sum(1)

In [154]:
model = MatrixFactorization(n_users, n_items, n_factors=20)
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), 
                            lr=1e-6) # learning rate

Embedding(100004, 20, sparse=True)


In [155]:
rows, cols = mat.nonzero()
print(rows)
print(cols)
p = np.random.permutation(len(rows))
print(p)
rows, cols = rows[p], cols[p]
print(rows)
print(cols)

for row, col in zip(*(rows, cols)):
    # Turn data into variables
    rating = Variable(torch.FloatTensor([mat[row, col]]))
    row = Variable(torch.LongTensor([np.long(row)]))
    col = Variable(torch.LongTensor([np.long(col)]))
    
    # Predict and calculate loss
    prediction = model(row, col)
    loss = loss_func(prediction, rating)
    
    # Backpropagate
    loss.backward()
    
    # Update the parameters
    optimizer.step()
    
    print(rating)
    print(row)
    print(col)
    print(prediction)
    print(loss)
  

[  0   0   0 ... 670 670 670]
[  30  833  859 ... 4597 4610 4696]
[61932 99903 93954 ... 46176 80075 43567]
[451 670 623 ... 333 546 310]
[1777  522 2159 ... 8412 5678  743]
tensor([3.])
tensor([451])
tensor([1777])
tensor([-6.7903], grad_fn=<SumBackward1>)
tensor(95.8505, grad_fn=<MseLossBackward>)
tensor([5.])
tensor([670])
tensor([522])
tensor([-8.0147], grad_fn=<SumBackward1>)
tensor(169.3819, grad_fn=<MseLossBackward>)
tensor([3.])
tensor([623])
tensor([2159])
tensor([-9.8004], grad_fn=<SumBackward1>)
tensor(163.8506, grad_fn=<MseLossBackward>)
tensor([3.])
tensor([160])
tensor([203])
tensor([3.6831], grad_fn=<SumBackward1>)
tensor(0.4667, grad_fn=<MseLossBackward>)
tensor([4.])
tensor([12])
tensor([3419])
tensor([5.1928], grad_fn=<SumBackward1>)
tensor(1.4227, grad_fn=<MseLossBackward>)
tensor([5.])
tensor([186])
tensor([284])
tensor([1.2577], grad_fn=<SumBackward1>)
tensor(14.0044, grad_fn=<MseLossBackward>)
tensor([2.])
tensor([188])
tensor([1216])
tensor([-2.4890], grad_fn=<Su

tensor(0.0839, grad_fn=<MseLossBackward>)
tensor([5.])
tensor([202])
tensor([1288])
tensor([-6.7020], grad_fn=<SumBackward1>)
tensor(136.9363, grad_fn=<MseLossBackward>)
tensor([3.])
tensor([194])
tensor([1690])
tensor([-5.7080], grad_fn=<SumBackward1>)
tensor(75.8286, grad_fn=<MseLossBackward>)
tensor([3.5000])
tensor([586])
tensor([3324])
tensor([6.1363], grad_fn=<SumBackward1>)
tensor(6.9502, grad_fn=<MseLossBackward>)
tensor([3.])
tensor([589])
tensor([529])
tensor([-1.6144], grad_fn=<SumBackward1>)
tensor(21.2927, grad_fn=<MseLossBackward>)
tensor([5.])
tensor([515])
tensor([472])
tensor([8.1037], grad_fn=<SumBackward1>)
tensor(9.6328, grad_fn=<MseLossBackward>)
tensor([4.])
tensor([238])
tensor([3278])
tensor([1.4973], grad_fn=<SumBackward1>)
tensor(6.2633, grad_fn=<MseLossBackward>)
tensor([5.])
tensor([194])
tensor([1036])
tensor([-2.4751], grad_fn=<SumBackward1>)
tensor(55.8778, grad_fn=<MseLossBackward>)
tensor([3.5000])
tensor([573])
tensor([6958])
tensor([1.4382], grad_fn=<

tensor([1.])
tensor([460])
tensor([1557])
tensor([-3.6720], grad_fn=<SumBackward1>)
tensor(21.8273, grad_fn=<MseLossBackward>)
tensor([3.5000])
tensor([234])
tensor([495])
tensor([-4.2895], grad_fn=<SumBackward1>)
tensor(60.6759, grad_fn=<MseLossBackward>)
tensor([1.])
tensor([379])
tensor([1129])
tensor([-4.5974], grad_fn=<SumBackward1>)
tensor(31.3310, grad_fn=<MseLossBackward>)
tensor([5.])
tensor([72])
tensor([8652])
tensor([0.7335], grad_fn=<SumBackward1>)
tensor(18.2033, grad_fn=<MseLossBackward>)
tensor([3.])
tensor([450])
tensor([1684])
tensor([-4.5056], grad_fn=<SumBackward1>)
tensor(56.3336, grad_fn=<MseLossBackward>)
tensor([4.])
tensor([513])
tensor([1089])
tensor([2.9774], grad_fn=<SumBackward1>)
tensor(1.0457, grad_fn=<MseLossBackward>)
tensor([1.])
tensor([261])
tensor([3156])
tensor([-3.7857], grad_fn=<SumBackward1>)
tensor(22.9034, grad_fn=<MseLossBackward>)
tensor([3.])
tensor([315])
tensor([6515])
tensor([-2.6860], grad_fn=<SumBackward1>)
tensor(32.3306, grad_fn=<Mse

KeyboardInterrupt: 